In [1]:
## CONFIG
model_version = "s2v"   # one of s2v, aida, or w2v
run_completion_experiment = False
run_ranking_experiment = True
exclude_minority_vectores = False
log_path = "/opt3/home/lofi/github/SimilarityAndAnalogy/analogy/btw17/log/"

In [2]:
import pandas as pd
import numpy as np

In [3]:
result_file_name=log_path+model_version+"_"+("NoMinority" if exclude_minority_vectores else "Minority")+".result.csv"

In [5]:
r_columns=('a1','a2','b1','b2','ags_rating','model_rating')
df=pd.read_csv(result_file_name, sep='\t', encoding='utf-8')
groups=df.groupby(by="a1")
